In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm import tqdm

# Fonctions d'activation
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def softmax(x):
    max_x = np.max(x, axis=1, keepdims=True)
    exp_x = np.exp(x - max_x)
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)



class Conv2D:
    def __init__(self, in_channels, out_channels, kernel_size):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size

        # Initialisation Glorot (Xavier)
        fan_in = in_channels * kernel_size * kernel_size
        fan_out = out_channels * kernel_size * kernel_size
        std = np.sqrt(2.0 / (fan_in + fan_out))
        self.weights = np.random.normal(0, std, (out_channels, in_channels, kernel_size, kernel_size))
        self.bias = np.zeros(out_channels)
        self.cache = None

    def forward(self, x):
        batch_size, in_height, in_width, _ = x.shape
        out_height = in_height - self.kernel_size + 1
        out_width = in_width - self.kernel_size + 1

        output = np.zeros((batch_size, out_height, out_width, self.out_channels))

        for i in range(out_height):
            for j in range(out_width):
                region = x[:, i:i+self.kernel_size, j:j+self.kernel_size, :]
                region_reshaped = region.reshape((batch_size, -1))
                weights_reshaped = self.weights.reshape((self.out_channels, -1))
                output[:, i, j, :] = region_reshaped @ weights_reshaped.T + self.bias

        self.cache = (x, (out_height, out_width))
        return output


    def backward(self, dout, lr):
        x, (out_height, out_width) = self.cache
        batch_size, in_height, in_width, in_channels = x.shape

        dweights = np.zeros_like(self.weights)
        dbias = np.sum(dout, axis=(0, 1, 2))
        dx = np.zeros_like(x)

        for i in range(out_height):
            i_start, i_end = i, i + self.kernel_size
            for j in range(out_width):
                j_start, j_end = j, j + self.kernel_size

                region = x[:, i_start:i_end, j_start:j_end, :]
                dout_slice = dout[:, i:i+1, j:j+1, :]

                # Correction : Calcul de dweights avec transposition
                for k in range(self.out_channels):
                    product = region * dout_slice[:, :, :, k:k+1]  # Broadcasting correct
                    temp = np.sum(product, axis=0)  # Forme (kernel_size, kernel_size, in_channels)
                    dweights[k] += temp.transpose(2, 0, 1)  # Transposée en (in_channels, kernel_size, kernel_size)

                # Correction : Calcul vectorisé de dx
                dout_slice_flat = dout_slice.reshape(batch_size, -1)  # Forme (batch_size, out_channels)
                weights_flat = self.weights.transpose(1, 2, 3, 0).reshape(-1, self.out_channels)
                dx_region = np.dot(dout_slice_flat, weights_flat.T).reshape(
                    batch_size, self.kernel_size, self.kernel_size, in_channels
                )
                dx[:, i_start:i_end, j_start:j_end, :] += dx_region

        # Mise à jour des paramètres
        self.weights -= lr * dweights / batch_size
        self.bias -= lr * dbias / batch_size

        return dx


class AvgPool2D:
    def __init__(self, pool_size=2, stride=2):
        self.pool_size = pool_size
        self.stride = stride
        self.cache = None

    def forward(self, x):
        batch_size, in_height, in_width, in_channels = x.shape
        out_height = (in_height - self.pool_size) // self.stride + 1
        out_width = (in_width - self.pool_size) // self.stride + 1

        output = np.zeros((batch_size, out_height, out_width, in_channels))

        for i in range(out_height):
            for j in range(out_width):
                h_start = i * self.stride
                h_end = h_start + self.pool_size
                w_start = j * self.stride
                w_end = w_start + self.pool_size

                region = x[:, h_start:h_end, w_start:w_end, :]
                output[:, i, j, :] = np.mean(region, axis=(1, 2))

        self.cache = (x.shape, region)
        return output

    def backward(self, dout):
        input_shape, _ = self.cache
        batch_size, out_height, out_width, in_channels = dout.shape
        dx = np.zeros(input_shape)
        pool_area = self.pool_size * self.pool_size

        for i in range(out_height):
            for j in range(out_width):
                h_start = i * self.stride
                h_end = h_start + self.pool_size
                w_start = j * self.stride
                w_end = w_start + self.pool_size

                grad = dout[:, i:i+1, j:j+1, :] / pool_area
                dx[:, h_start:h_end, w_start:w_end, :] += grad

        return dx

class Flatten:
    def __init__(self):
        self.cache = None

    def forward(self, x):
        self.cache = x.shape
        return x.reshape(x.shape[0], -1)

    def backward(self, dout):
        return dout.reshape(self.cache)

class Dense:
    def __init__(self, input_size, output_size):
        # Initialisation Glorot (Xavier)
        std = np.sqrt(2.0 / (input_size + output_size))
        self.weights = np.random.normal(0, std, (input_size, output_size))
        self.bias = np.zeros(output_size)
        self.cache = None

    def forward(self, x):
        self.cache = x
        return np.dot(x, self.weights) + self.bias

    def backward(self, dout, lr):
        x = self.cache
        batch_size = x.shape[0]

        dw = np.dot(x.T, dout)
        db = np.sum(dout, axis=0)
        dx = np.dot(dout, self.weights.T)

        self.weights -= lr * dw / batch_size
        self.bias -= lr * db / batch_size

        return dx

class ReLU:
    def __init__(self):
        self.cache = None

    def forward(self, x):
        self.cache = x
        return relu(x)

    def backward(self, dout):
        x = self.cache
        dx = dout * relu_derivative(x)
        return dx

class Softmax:
    def __init__(self):
        self.cache = None

    def forward(self, x):
        output = softmax(x)
        self.cache = output
        return output

    def backward(self, dout, y_true):
        output = self.cache
        batch_size = y_true.shape[0]
        #dx = (output - y_true) / batch_size
        dx = (output - y_true)

        return dx

# ===========================================
# Architecture LeNet-5 améliorée
# ===========================================
class LeNet5:
    def __init__(self, input_shape=(32, 32, 1), num_classes=33):

        self.layers = [
            Conv2D(input_shape[-1], 6, 5),  # C1: 6 filtres 5x5 -> 28x28x6
            ReLU(),
            AvgPool2D(2, 2),                # S2: Pooling -> 14x14x6
            Conv2D(6, 16, 5),               # C3: 16 filtres 5x5 -> 10x10x16
            ReLU(),
            AvgPool2D(2, 2),                # S4: Pooling -> 5x5x16
            Flatten(),
            Dense(16*5*5, 120),             # C5: Fully connected 120 neurones
            ReLU(),
            Dense(120, 84),                 # F6: Fully connected 84 neurones
            ReLU(),
            Dense(84, num_classes),         # Sortie: 33 classes
            Softmax()
        ]

    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def backward(self, y_pred, y_true, lr):
        grad = self.layers[-1].backward(None, y_true)
        for layer in reversed(self.layers[:-1]):
            if isinstance(layer, (Conv2D, Dense)):
                grad = layer.backward(grad, lr)
            else:
                grad = layer.backward(grad)
        return grad

    def predict(self, x):
        return self.forward(x).argmax(axis=1)

# ===========================================
# Fonctions utilitaires pour l'entraînement
# ===========================================
def cross_entropy_loss(y_pred, y_true):
    m = y_true.shape[0]
    y_pred = np.clip(y_pred, 1e-15, 1-1e-15)
    log_likelihood = -np.sum(y_true * np.log(y_pred)) / m
    return log_likelihood
    #correct_logprobs = -np.log(y_pred[np.arange(m), y_true.argmax(axis=1)])
    #return np.sum(correct_logprobs) / m

def accuracy(y_pred, y_true):
    return np.mean(y_pred.argmax(axis=1) == y_true.argmax(axis=1))

def train_model(model, X_train, y_train, X_val, y_val, epochs=10, batch_size=32, lr=0.01):
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    n_batches = int(np.ceil(X_train.shape[0] / batch_size))

    for epoch in range(epochs):
        start_time = time.time()
        epoch_loss = 0
        epoch_acc = 0

        # Mélanger les données
        indices = np.arange(X_train.shape[0])
        np.random.shuffle(indices)
        X_train_shuffled = X_train[indices]
        y_train_shuffled = y_train[indices]

        # Barre de progression pour les batches
        for i in tqdm(range(n_batches), desc=f"Epoch {epoch+1}/{epochs}"):
            start = i * batch_size
            end = min((i+1) * batch_size, X_train.shape[0])
            X_batch = X_train_shuffled[start:end]
            y_batch = y_train_shuffled[start:end]

            # Propagation avant
            y_pred = model.forward(X_batch)

            # Calcul de la perte et précision
            loss = cross_entropy_loss(y_pred, y_batch)
            acc = accuracy(y_pred, y_batch)

            # Rétropropagation
            model.backward(y_pred, y_batch, lr)

            epoch_loss += loss
            epoch_acc += acc

        # Calcul des moyennes
        epoch_loss /= n_batches
        epoch_acc /= n_batches

        # Validation
        val_pred = model.forward(X_val)
        val_loss = cross_entropy_loss(val_pred, y_val)
        val_acc = accuracy(val_pred, y_val)

        # Stockage des résultats
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        epoch_time = time.time() - start_time
        print(f"Epoch {epoch+1}/{epochs} - {epoch_time:.2f}s - "
              f"Train Loss: {epoch_loss:.4f} - Train Acc: {epoch_acc:.4f} - "
              f"Val Loss: {val_loss:.4f} - Val Acc: {val_acc:.4f}")

    return train_losses, val_losses, train_accuracies, val_accuracies

# ===========================================
# Chargement et prétraitement des données
# ===========================================
def load_and_preprocess_data(data_dir):
    csv_path = os.path.join(data_dir, 'labels-map.csv')

    try:
        labels_df = pd.read_csv(csv_path, header=None, names=['image_path', 'label'])
        print("labels-map.csv found. Loading data from CSV...")

        # labels_df['image_path'] = labels_df['image_path'].str.replace(
        #     './images-data-64/tifinagh-images/',
        #     'tifinagh-images/',
        #     regex=False
        # )
        labels_df['image_path'] = labels_df['image_path'].str.replace(
            './images-data-64/tifinagh-images/',
            '',  # Supprimer complètement le préfixe erroné
            regex=False
        )
        labels_df['image_path'] = os.path.join(data_dir, 'tifinagh-images') + '/' + labels_df['image_path']


        labels_df['image_path'] = labels_df['image_path'].str.replace('./', '', regex=False)
        labels_df['image_path'] = labels_df['image_path'].apply(
            lambda x: os.path.join(data_dir, x)
        )
    except FileNotFoundError:
        print("labels-map.csv not found. Building DataFrame from folders...")
        base_dir = os.path.join(data_dir, 'tifinagh-images')
        image_paths = []
        labels = []
        for label_dir in os.listdir(base_dir):
            label_path = os.path.join(base_dir, label_dir)
            if os.path.isdir(label_path):
                for img_name in os.listdir(label_path):
                    if img_name.lower().endswith(('.jpeg', '.jpg', '.png')):
                        image_paths.append(os.path.join(label_path, img_name))
                        labels.append(label_dir)
        labels_df = pd.DataFrame({'image_path': image_paths, 'label': labels})

    assert not labels_df.empty, "No data loaded. Check dataset files."
    print(f"Loaded {len(labels_df)} samples with {labels_df['label'].nunique()} unique classes.")

    label_encoder = LabelEncoder()
    labels_df['label_encoded'] = label_encoder.fit_transform(labels_df['label'])
    num_classes = len(label_encoder.classes_)


    def load_and_preprocess_image(image_path, target_size=(32, 32)):
      try:
          # Vérification renforcée de l'existence du fichier
          if not os.path.exists(image_path):
              # Tentative de récupération alternative
              alt_path = image_path.replace('tifinagh-images/', '')
              if os.path.exists(alt_path):
                  image_path = alt_path
                  print(f"Info: Using alternative path - {alt_path}")
              else:
                  print(f"Warning: File not found - {image_path}")
                  return None

          # Chargement de l'image
          img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
          if img is None:
              print(f"Warning: Failed to load image - {image_path}")
              return None

          # Prétraitement de l'image
          img = cv2.resize(img, target_size)
          img = img.astype(np.float32) / 255.0

          # Normalisation: soustraction de la moyenne et division par l'écart-type
          img_mean = np.mean(img)
          img_std = np.std(img)

          # Éviter la division par zéro
          if img_std < 1e-7:
              img_std = 1.0

          img = (img - img_mean) / img_std
          return img.reshape(32, 32, 1)

      except Exception as e:
          print(f"Error processing image {image_path}: {str(e)}")
          return None

    images = []
    valid_indices = []

    for idx, path in enumerate(labels_df['image_path']):
        img = load_and_preprocess_image(path)
        if img is not None:
            images.append(img)
            valid_indices.append(idx)

    valid_df = labels_df.iloc[valid_indices]
    X = np.array(images)
    y = valid_df['label_encoded'].values

    assert X.shape[0] == y.shape[0], "Mismatch between number of images and labels"
    assert X.shape[1:] == (32, 32, 1), f"Expected image shape (32,32,1), got {X.shape[1:]}"

    # Division stratifiée des données
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=42
    )

    one_hot_encoder = OneHotEncoder(sparse_output=False)
    y_train_one_hot = one_hot_encoder.fit_transform(y_train.reshape(-1, 1))
    y_val_one_hot = one_hot_encoder.transform(y_val.reshape(-1, 1))
    y_test_one_hot = one_hot_encoder.transform(y_test.reshape(-1, 1))

    return X_train, X_val, X_test, y_train_one_hot, y_val_one_hot, y_test_one_hot, num_classes, label_encoder

# ===========================================
# Visualisation des résultats
# ===========================================
def plot_training_history(train_losses, val_losses, train_accuracies, val_accuracies):
    plt.figure(figsize=(15, 6))

    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Loss Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label='Train Accuracy')
    plt.plot(val_accuracies, label='Validation Accuracy')
    plt.title('Accuracy Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.savefig('lenet5_training_curves.png')
    plt.show()

def plot_confusion_matrix(y_true, y_pred, class_names):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(15, 12))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names,
                yticklabels=class_names)
    plt.title('Confusion Matrix (Test set)')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig('lenet5_confusion_matrix.png')
    plt.show()

# ===========================================
# Point d'entrée principal
# ===========================================
if __name__ == "__main__":
    #data_dir = os.path.join(os.getcwd(), 'amhcd-data-64')
    data_dir = os.path.join(os.getcwd(), 'amhcd-data/amhcd-data-64')





    # Charger les données
    X_train, X_val, X_test, y_train, y_val, y_test, num_classes, label_encoder = load_and_preprocess_data(data_dir)

    print(f"Train: {X_train.shape[0]} samples, Validation: {X_val.shape[0]} samples, Test: {X_test.shape[0]} samples")
    print(f"Image shape: {X_train.shape[1:]}, Number of classes: {num_classes}")

    # Réduction stratifiée pour le débogage (optionnel)
    # from sklearn.model_selection import train_test_split
    # _, X_train, _, y_train = train_test_split(X_train, y_train, train_size=2000, stratify=np.argmax(y_train, axis=1), random_state=42)
    # _, X_val, _, y_val = train_test_split(X_val, y_val, train_size=500, stratify=np.argmax(y_val, axis=1), random_state=42)

    # Initialiser le modèle
    model = LeNet5(input_shape=(32, 32, 1), num_classes=num_classes)

    # Entraîner le modèle avec un learning rate adaptatif
    train_losses, val_losses, train_accuracies, val_accuracies = train_model(
        model, X_train, y_train, X_val, y_val,
        epochs=20,
        batch_size=64,
        lr=0.001
    )

    # Visualiser les courbes
    plot_training_history(train_losses, val_losses, train_accuracies, val_accuracies)

    # Évaluation sur le test set
    print("Evaluating on test set...")
    y_pred_probs = model.forward(X_test)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(y_test, axis=1)

    test_acc = accuracy(y_pred_probs, y_test)
    print(f"\nTest Accuracy: {test_acc:.4f}")

    # Matrice de confusion
    plot_confusion_matrix(y_true, y_pred, label_encoder.classes_)

    # Rapport de classification
    print("\nClassification Report (Test set):")
    print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))

    # Visualisation des feature maps
    try:
        sample_idx = np.random.randint(0, len(X_test))
        sample_img = X_test[sample_idx]

        activations = sample_img[np.newaxis, ...]
        feature_maps = model.layers[0].forward(activations)

        plt.figure(figsize=(12, 6))
        plt.suptitle("Feature Maps: First Convolutional Layer", fontsize=16)
        for i in range(min(16, feature_maps.shape[-1])):
            plt.subplot(4, 4, i+1)
            plt.imshow(feature_maps[0, :, :, i], cmap='viridis')
            plt.axis('off')
        plt.tight_layout()
        plt.savefig('lenet5_feature_maps.png')
        plt.show()
    except Exception as e:
        print(f"Feature map visualization skipped: {e}")

In [1]:
##code 2

In [ ]:
!pip uninstall -y cupy-cuda11x cupy-cuda12x
!pip install cupy-cuda12x

from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
import numpy as np
import cupy as cp
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm import tqdm

# Fonctions d'activation
def relu(x):
    return cp.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(cp.float32)

def softmax(x):
    max_x = cp.max(x, axis=1, keepdims=True)
    exp_x = cp.exp(x - max_x)
    return exp_x / cp.sum(exp_x, axis=1, keepdims=True)

# Classes des couches
class Conv2D:
    def __init__(self, in_channels, out_channels, kernel_size):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size

        # Initialisation avec CuPy
        fan_in = in_channels * kernel_size * kernel_size
        fan_out = out_channels * kernel_size * kernel_size
        std = cp.sqrt(2.0 / (fan_in + fan_out))
        self.weights = cp.random.normal(0, std, (out_channels, in_channels, kernel_size, kernel_size))
        self.bias = cp.zeros(out_channels)
        self.cache = None

    def forward(self, x):
        batch_size, in_height, in_width, _ = x.shape
        out_height = in_height - self.kernel_size + 1
        out_width = in_width - self.kernel_size + 1

        output = cp.zeros((batch_size, out_height, out_width, self.out_channels))

        for i in range(out_height):
            for j in range(out_width):
                region = x[:, i:i+self.kernel_size, j:j+self.kernel_size, :]
                region_reshaped = region.reshape((batch_size, -1))
                weights_reshaped = self.weights.reshape((self.out_channels, -1))
                output[:, i, j, :] = region_reshaped @ weights_reshaped.T + self.bias

        self.cache = (x, (out_height, out_width))
        return output

    def backward(self, dout, lr):
        x, (out_height, out_width) = self.cache
        batch_size, in_height, in_width, in_channels = x.shape

        dweights = cp.zeros_like(self.weights)
        dbias = cp.sum(dout, axis=(0, 1, 2))
        dx = cp.zeros_like(x)

        for i in range(out_height):
            i_start, i_end = i, i + self.kernel_size
            for j in range(out_width):
                j_start, j_end = j, j + self.kernel_size

                region = x[:, i_start:i_end, j_start:j_end, :]
                dout_slice = dout[:, i:i+1, j:j+1, :]

                for k in range(self.out_channels):
                    product = region * dout_slice[:, :, :, k:k+1]
                    temp = cp.sum(product, axis=0)
                    dweights[k] += temp.transpose(2, 0, 1)

                dout_slice_flat = dout_slice.reshape(batch_size, -1)
                weights_flat = self.weights.transpose(1, 2, 3, 0).reshape(-1, self.out_channels)
                dx_region = dout_slice_flat @ weights_flat.T
                dx_region = dx_region.reshape(batch_size, self.kernel_size, self.kernel_size, in_channels)
                dx[:, i_start:i_end, j_start:j_end, :] += dx_region

        self.weights -= lr * dweights / batch_size
        self.bias -= lr * dbias / batch_size

        return dx

class AvgPool2D:
    def __init__(self, pool_size=2, stride=2):
        self.pool_size = pool_size
        self.stride = stride
        self.cache = None

    def forward(self, x):
        batch_size, in_height, in_width, in_channels = x.shape
        out_height = (in_height - self.pool_size) // self.stride + 1
        out_width = (in_width - self.pool_size) // self.stride + 1

        output = cp.zeros((batch_size, out_height, out_width, in_channels))

        for i in range(out_height):
            for j in range(out_width):
                h_start = i * self.stride
                h_end = h_start + self.pool_size
                w_start = j * self.stride
                w_end = w_start + self.pool_size

                region = x[:, h_start:h_end, w_start:w_end, :]
                output[:, i, j, :] = cp.mean(region, axis=(1, 2))

        self.cache = (x.shape, region)
        return output

    def backward(self, dout):
        input_shape, _ = self.cache
        batch_size, out_height, out_width, in_channels = dout.shape
        dx = cp.zeros(input_shape)
        pool_area = self.pool_size * self.pool_size

        for i in range(out_height):
            for j in range(out_width):
                h_start = i * self.stride
                h_end = h_start + self.pool_size
                w_start = j * self.stride
                w_end = w_start + self.pool_size

                grad = dout[:, i:i+1, j:j+1, :] / pool_area
                dx[:, h_start:h_end, w_start:w_end, :] += grad

        return dx

class Flatten:
    def __init__(self):
        self.cache = None

    def forward(self, x):
        self.cache = x.shape
        return x.reshape(x.shape[0], -1)

    def backward(self, dout):
        return dout.reshape(self.cache)

class Dense:
    def __init__(self, input_size, output_size):
        std = cp.sqrt(2.0 / (input_size + output_size))
        self.weights = cp.random.normal(0, std, (input_size, output_size))
        self.bias = cp.zeros(output_size)
        self.cache = None

    def forward(self, x):
        self.cache = x
        return cp.dot(x, self.weights) + self.bias

    def backward(self, dout, lr):
        x = self.cache
        batch_size = x.shape[0]

        dw = cp.dot(x.T, dout)
        db = cp.sum(dout, axis=0)
        dx = cp.dot(dout, self.weights.T)

        self.weights -= lr * dw / batch_size
        self.bias -= lr * db / batch_size

        return dx

class ReLU:
    def __init__(self):
        self.cache = None

    def forward(self, x):
        self.cache = x
        return relu(x)

    def backward(self, dout):
        x = self.cache
        dx = dout * relu_derivative(x)
        return dx

class Softmax:
    def __init__(self):
        self.cache = None

    def forward(self, x):
        output = softmax(x)
        self.cache = output
        return output

    def backward(self, dout, y_true):
        output = self.cache
        batch_size = y_true.shape[0]
        dx = (output - y_true)
        return dx

# Architecture LeNet-5
class LeNet5:
    def __init__(self, input_shape=(32, 32, 1), num_classes=33):
        self.layers = [
            Conv2D(input_shape[-1], 6, 5),
            ReLU(),
            AvgPool2D(2, 2),
            Conv2D(6, 16, 5),
            ReLU(),
            AvgPool2D(2, 2),
            Flatten(),
            Dense(16*5*5, 120),
            ReLU(),
            Dense(120, 84),
            ReLU(),
            Dense(84, num_classes),
            Softmax()
        ]

    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def backward(self, y_pred, y_true, lr):
        grad = self.layers[-1].backward(None, y_true)
        for layer in reversed(self.layers[:-1]):
            if isinstance(layer, (Conv2D, Dense)):
                grad = layer.backward(grad, lr)
            else:
                grad = layer.backward(grad)
        return grad

    def predict(self, x):
        return self.forward(x).argmax(axis=1)

# Fonctions utilitaires
def cross_entropy_loss(y_pred, y_true):
    m = y_true.shape[0]
    y_pred = cp.clip(y_pred, 1e-15, 1-1e-15)
    log_likelihood = -cp.sum(y_true * cp.log(y_pred)) / m
    return log_likelihood

def accuracy(y_pred, y_true):
    return cp.mean(y_pred.argmax(axis=1) == y_true.argmax(axis=1))

def train_model(model, X_train, y_train, X_val, y_val, epochs=10, batch_size=32, lr=0.01):
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    n_batches = int(cp.ceil(X_train.shape[0] / batch_size))

    for epoch in range(epochs):
        start_time = time.time()
        epoch_loss = 0
        epoch_acc = 0

        indices = cp.arange(X_train.shape[0])
        cp.random.shuffle(indices)
        X_train_shuffled = X_train[indices]
        y_train_shuffled = y_train[indices]

        for i in tqdm(range(n_batches), desc=f"Epoch {epoch+1}/{epochs}"):
            start = i * batch_size
            end = min((i+1) * batch_size, X_train.shape[0])
            X_batch = X_train_shuffled[start:end]
            y_batch = y_train_shuffled[start:end]

            y_pred = model.forward(X_batch)

            loss = cross_entropy_loss(y_pred, y_batch)
            acc = accuracy(y_pred, y_batch)

            model.backward(y_pred, y_batch, lr)

            epoch_loss += loss
            epoch_acc += acc

        epoch_loss /= n_batches
        epoch_acc /= n_batches

        val_pred = model.forward(X_val)
        val_loss = cross_entropy_loss(val_pred, y_val)
        val_acc = accuracy(val_pred, y_val)

        train_losses.append(float(epoch_loss))
        train_accuracies.append(float(epoch_acc))
        val_losses.append(float(val_loss))
        val_accuracies.append(float(val_acc))

        epoch_time = time.time() - start_time
        print(f"Epoch {epoch+1}/{epochs} - {epoch_time:.2f}s - "
              f"Train Loss: {float(epoch_loss):.4f} - Train Acc: {float(epoch_acc):.4f} - "
              f"Val Loss: {float(val_loss):.4f} - Val Acc: {float(val_acc):.4f}")

    return train_losses, val_losses, train_accuracies, val_accuracies

# Chargement des données
def load_and_preprocess_data(data_dir):
    csv_path = os.path.join(data_dir, 'labels-map.csv')

    try:
        labels_df = pd.read_csv(csv_path, header=None, names=['image_path', 'label'])
        print("labels-map.csv found. Loading data from CSV...")

        labels_df['image_path'] = labels_df['image_path'].str.replace(
            './images-data-64/tifinagh-images/',
            '',
            regex=False
        )
        labels_df['image_path'] = os.path.join(data_dir, 'tifinagh-images') + '/' + labels_df['image_path']
        labels_df['image_path'] = labels_df['image_path'].str.replace('./', '', regex=False)
    except FileNotFoundError:
        print("labels-map.csv not found. Building DataFrame from folders...")
        base_dir = os.path.join(data_dir, 'tifinagh-images')

        # Vérification de l'existence du dossier
        if not os.path.exists(base_dir):
            print(f"Error: Directory '{base_dir}' does not exist!")
            print(f"Available directories in {data_dir}: {os.listdir(data_dir)}")
            raise FileNotFoundError(f"Directory '{base_dir}' not found")

        image_paths = []
        labels = []
        for label_dir in os.listdir(base_dir):
            label_path = os.path.join(base_dir, label_dir)
            if os.path.isdir(label_path):
                for img_name in os.listdir(label_path):
                    if img_name.lower().endswith(('.jpeg', '.jpg', '.png')):
                        image_paths.append(os.path.join(label_path, img_name))
                        labels.append(label_dir)
        labels_df = pd.DataFrame({'image_path': image_paths, 'label': labels})

    assert not labels_df.empty, "No data loaded. Check dataset files."
    print(f"Loaded {len(labels_df)} samples with {labels_df['label'].nunique()} unique classes.")

    label_encoder = LabelEncoder()
    labels_df['label_encoded'] = label_encoder.fit_transform(labels_df['label'])
    num_classes = len(label_encoder.classes_)

    def load_and_preprocess_image(image_path, target_size=(32, 32)):
        try:
            if not os.path.exists(image_path):
                alt_path = image_path.replace('tifinagh-images/', '')
                if os.path.exists(alt_path):
                    image_path = alt_path
                else:
                    print(f"Warning: File not found - {image_path}")
                    return None

            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"Warning: Failed to load image - {image_path}")
                return None

            img = cv2.resize(img, target_size)
            img = img.astype(np.float32) / 255.0

            img_mean = np.mean(img)
            img_std = np.std(img)
            if img_std < 1e-7:
                img_std = 1.0

            img = (img - img_mean) / img_std
            return img.reshape(32, 32, 1)

        except Exception as e:
            print(f"Error processing image {image_path}: {str(e)}")
            return None

    images = []
    valid_indices = []

    for idx, path in enumerate(labels_df['image_path']):
        img = load_and_preprocess_image(path)
        if img is not None:
            images.append(img)
            valid_indices.append(idx)

    valid_df = labels_df.iloc[valid_indices]
    X = np.array(images)
    y = valid_df['label_encoded'].values

    assert X.shape[0] == y.shape[0], "Mismatch between number of images and labels"
    assert X.shape[1:] == (32, 32, 1), f"Expected image shape (32,32,1), got {X.shape[1:]}"

    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=42
    )

    one_hot_encoder = OneHotEncoder(sparse_output=False)
    y_train_one_hot = one_hot_encoder.fit_transform(y_train.reshape(-1, 1))
    y_val_one_hot = one_hot_encoder.transform(y_val.reshape(-1, 1))
    y_test_one_hot = one_hot_encoder.transform(y_test.reshape(-1, 1))

    X_train = cp.asarray(X_train)
    X_val = cp.asarray(X_val)
    X_test = cp.asarray(X_test)
    y_train_one_hot = cp.asarray(y_train_one_hot)
    y_val_one_hot = cp.asarray(y_val_one_hot)
    y_test_one_hot = cp.asarray(y_test_one_hot)

    return X_train, X_val, X_test, y_train_one_hot, y_val_one_hot, y_test_one_hot, num_classes, label_encoder

# Visualisation
def plot_training_history(train_losses, val_losses, train_accuracies, val_accuracies):
    plt.figure(figsize=(15, 6))

    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Loss Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label='Train Accuracy')
    plt.plot(val_accuracies, label='Validation Accuracy')
    plt.title('Accuracy Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.savefig('lenet5_training_curves.png')
    plt.show()

def plot_confusion_matrix(y_true, y_pred, class_names):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(15, 12))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names,
                yticklabels=class_names)
    plt.title('Confusion Matrix (Test set)')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig('lenet5_confusion_matrix.png')
    plt.show()

# Point d'entrée principal
if __name__ == "__main__":


    data_dir = '/content/amhcd-data/amhcd-data-64'

    # Vérifiez si le dossier existe
    if not os.path.exists(data_dir):
        print(f"Data directory not found: {data_dir}")
        print("Available directories in /content/drive/MyDrive:")
        print(os.listdir('/content/drive/MyDrive'))
        raise FileNotFoundError(f"Directory not found: {data_dir}")

    print(f"Loading data from: {data_dir}")
    print(f"Directory contents: {os.listdir(data_dir)}")

    # Charger les données
    X_train, X_val, X_test, y_train, y_val, y_test, num_classes, label_encoder = load_and_preprocess_data(data_dir)

    print(f"Train: {X_train.shape[0]} samples, Validation: {X_val.shape[0]} samples, Test: {X_test.shape[0]} samples")
    print(f"Image shape: {X_train.shape[1:]}, Number of classes: {num_classes}")

    # Initialiser le modèle
    model = LeNet5(input_shape=(32, 32, 1), num_classes=num_classes)

    # Entraîner le modèle
    train_losses, val_losses, train_accuracies, val_accuracies = train_model(
        model, X_train, y_train, X_val, y_val,
        epochs=20,
        batch_size=64,
        lr=0.01
    )

    # Visualiser les courbes
    plot_training_history(train_losses, val_losses, train_accuracies, val_accuracies)

    # Évaluation sur le test set
    print("Evaluating on test set...")
    y_pred_probs = model.forward(X_test)
    y_pred = y_pred_probs.argmax(axis=1).get()
    y_true = y_test.argmax(axis=1).get()

    test_acc = (y_pred == y_true).mean()
    print(f"\nTest Accuracy: {test_acc:.4f}")

    # Matrice de confusion
    plot_confusion_matrix(y_true, y_pred, label_encoder.classes_)

    # Rapport de classification
    print("\nClassification Report (Test set):")
    print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))

    # Visualisation des feature maps
    try:
        sample_idx = cp.random.randint(0, len(X_test))
        sample_img = X_test[sample_idx: sample_idx+1]

        activations = sample_img
        feature_maps = model.layers[0].forward(activations).get()

        plt.figure(figsize=(12, 6))
        plt.suptitle("Feature Maps: First Convolutional Layer", fontsize=16)
        for i in range(min(16, feature_maps.shape[-1])):
            plt.subplot(4, 4, i+1)
            plt.imshow(feature_maps[0, :, :, i], cmap='viridis')
            plt.axis('off')
        plt.tight_layout()
        plt.savefig('lenet5_feature_maps.png')
        plt.show()
    except Exception as e:
        print(f"Feature map visualization skipped: {e}")

In [ ]:
##pour voir drive !
# Remplacer cette partie
if __name__ == "__main__":
    drive.mount('/content/drive')

    # Afficher la structure des dossiers
    print("Contenu de /content/drive/MyDrive :")
    print(os.listdir('/content/drive/MyDrive'))

    # Trouver le bon chemin
    data_dir = None
    possible_paths = [
        '/content/drive/MyDrive/amhcd-data-64',
        '/content/drive/MyDrive/amhcd-data/amhcd-data-64',
        '/content/drive/MyDrive/Datasets/amhcd-data-64',
        # Ajoutez d'autres chemins possibles ici
    ]

    for path in possible_paths:
        if os.path.exists(path):
            data_dir = path
            break

    if not data_dir:
        print("Dossier non trouvé. Veuillez vérifier le chemin dans Google Drive")
        print("Contenu complet de /content/drive/MyDrive :")
